# Tutorial 3: Automating Hunting Procedures

## Objectives

- Bootstrap threat hunting procedure notebook from template
- Populate new procedure notebook with canned language and desired search queries
- Enumerate available hunting procedures
- Launch an instance of the new hunting procedure

![hunting-create-notebook](images/hunting-create-notebook.png)

## Description

This tutorial shows how to create a well-formed threat hunting procedure notebook using a provided template.
A threat hunting procedure is a Jupyter notebook intended to be re-used and shared with other analysts.

## Step 1: Read Procedure Template

> Before we begin, please ensure that the notebook kernel is set to `taegis-hunting-tutorials`.

Unlike notebooks used for free-form exploratory data analysis, a threat hunting procedure should contain:

- Notebook metadata to help organize the procedure in a catalog of hunting procedures kept under version control.
- Markdown content to describe the threat and the methodology used to find evidence of the threat. This will be used to populate the key findings section of a Taegis investigation.
- Code content to import useful dependencies and create a Taegis investigation from the notebook.
- Taegis queries to fetch relevant data in alerts and/or events.
- Analysis logic, usually in the form of filtering and transforming `pandas` DataFrames.

We find it easier to create a new hunting procedure from a template.
The procedure author only needs to fill in the details specific to the new procedure, rather than worry about the other elements.

We provided a minimal procedure notebook template in `templates/procedure-template.ipynb`.
This template is a skeleton that we will fill in to make our new hunting procedure. We will call our new hunting procedure "My First Hunting Procedure".

In [1]:
import nbformat

procedure_template: nbformat.NotebookNode = nbformat.read(
    "templates/procedure-template.ipynb", as_version=nbformat.current_nbformat
)

## Step 2: Add Notebook Metadata

To make searching for hunting procedures easier in the future, let's add notebook-level metadata to the new hunting procedure.
While the `jupyter.kernelspec` metadata is defined in the Jupyter specification, the `hunting` metadata is entirely arbitrary and can be adapted for other organizational schemes.

![hunting-nb-metadata](images/hunting-notebook-metadata.png)

In [2]:
new_procedure_title = "My First Hunting Procedure"
output_filename = "tutorial-03-my-first-hunting-procedure.ipynb"

procedure_template.metadata.hunting = {
    "title": new_procedure_title,
    "attack_technique_ids": ["TXXXX.XXX"],
    "id": "tutorial-03-my-first-hunting-procedure",
    "description": "Here is my first hunting procedure.",
    "data_sources": ["scwx.process", "scwx.auth"],
    "tags": ["tutorial"],
}

Now that we configured the metadata, we can save our new procedure back into the templates directory:

In [3]:
procedure_template.cells = [
    nbformat.v4.new_markdown_cell(source=f"# {new_procedure_title}"),
    *procedure_template.cells
]
nbformat.validate(procedure_template, relax_add_props=True)
nbformat.write(procedure_template, f"templates/{output_filename}")

Let's confirm that the metadata was set correctly by reading all notebooks from the templates directory that contain hunting-related metadata:

In [4]:
import pandas as pd
from pathlib import Path

template_directory = Path("templates")
available_notebooks = pd.DataFrame(
    {
        "notebook_path": notebook_path,
        **nb.metadata.hunting,
    }
    for notebook_path in template_directory.glob("*.ipynb")
    if (nb := nbformat.read(notebook_path, as_version=nbformat.NO_CONVERT))
    if "hunting" in nb.metadata
)

available_notebooks.head()

,notebook_path,attack_technique_ids,data_sources,description,id,tags,title
0,templates\tutorial-03-my-first-hunting-procedu...,[TXXXX.XXX],"[scwx.process, scwx.auth]",Here is my first hunting procedure.,tutorial-03-my-first-hunting-procedure,[tutorial],My First Hunting Procedure
1,templates\tutorial-04-windows-service-manipula...,[T1562.001],[scwx.process],The goal of this procedure is to identify poss...,tutorial-04-example-procedure-abcd123,[windows],Suspicious Windows Service Manipulation


## Step 3: Populate Content

Navigate into the templates directory and open up our new notebook: `templates/tutorial-03-my-first-hunting-procedure.ipynb`.

**Markdown Content**

You should see markdown cells corresponding to the various sections of the report, such as executive summary, findings, and recommendations.
The content of those markdown cells will appear in the generated Taegis investigation.
When creating a hunting procedure, it is helpful to create a test investigation to ensure that the key findings are well-formed.

*Fill out each markdown section with the appropriate canned language.*

**Code Content**

You should also see a number of cells under the header "Step N: Step Title".
Each step in a hunting procedure usually contains the same components: some descriptive text, a query, some analysis of the query results, and a Taegis Magic command to stage relevant evidence to the (future) Taegis investigation.
A hunting procedure should be able to executable all steps from start to finish without raising exceptions.

*Fill out the steps with the queries and logic from the previous tutorial, adding descriptive text as appropriate.*

> We provided a pre-populated version of the Windows service manipulation hunting procedure in the templates directory for your convenience.

## Step 4: Commit to Version Control

Instructions on how to commit to version control are outside the scope of this tutorial.
Regardless of the version control system, remember to clear all cell outputs before committing.
Threat hunters should pull down the latest version of any hunting procedure from version control before running it.

## Step 5: Launch Instance of Hunting Procedure

To run our newly created hunting procedure manually, we can use `papermill` to create an instance of the notebook with the appropriate parameters.
`papermill` will read the template that we just created, inject several variables, and save a parameterized copy of the notebook to a new location.

> In the next tutorial, we will show how to use `papermill` as a Python module to parameterize and execute multiple notebooks concurrently.

In [5]:
!papermill \
--inject-paths \
--prepare-only \
templates/tutorial-03-my-first-hunting-procedure.ipynb \
output/instance-of-my-first-hunting-procedure.ipynb \
-p TAEGIS_TENANT_ID "145483" \
-p TAEGIS_ENVIRONMENT "foxtrot" \
-p INVESTIGATION_TITLE "My First Hunting Procedure"

Input Notebook:  templates/tutorial-03-my-first-hunting-procedure.ipynb
Output Notebook: output/instance-of-my-first-hunting-procedure.ipynb
Notebook JSON is invalid: Additional properties are not allowed ('id' was unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'id': 'a81e01b8',
 'metadata': {},
 'source': '# My First Hunting Procedure'}
Black is not installed, parameters wont be formatted


You can now open up `output/instance-of-my-first-hunting-procedure.ipynb` to see a parameterized instance of your new hunting procedure!

## Wrap-Up

In this tutorial, we did the following:

- Bootstraped a new threat hunting procedure notebook from a template
- Populated new procedure notebook with canned markdown and code content
- Demonstrated how to list available hunting procedures
- Launched a parameterized instance of the notebook using `papermill`